In [14]:
from dash import Dash, html, dcc, Input, Output, callback, Patch, ALL, State, ctx

import dash_cytoscape as cyto
from matplotlib import style
import json
import base64

app = Dash()

#Get the ID for the next node to be added to the list
#Goes through all ids and if they are integer finds the highest value and adds 1 to it, if none are integers returns 0. 
def getNextNodeID(elements):
    idList = []
    #print(elements)
    for element in elements:
        #print(element, ' element')
        if 'id' in element['data'].keys():
            if (element['data']['id'].isdigit()):
                idList.append(int(element['data']['id']))
    if len(idList) == 0:
        return 1
    else:
        return max(idList) + 1   
    
def addNewNode(elements):
    nodeDict = {}
    dataDict = {}
    nodeDict['id'] = str(getNextNodeID(elements))
    nodeDict['label'] = str(getNextNodeID(elements)) + ' seq'
    nodeDict['function'] = "nseq"
    dataDict['data'] = nodeDict
    dataDict['classes'] = 'default'
    #print(elements, ' addNode ', elements.append(dataDict))
    elements.append(dataDict)
    return elements

def addNewNodeNumber(elements, value):
    nodeDict = {}
    dataDict = {}
    nodeDict['id'] = str(value)
    nodeDict['label'] = str(value) + ' seq'
    nodeDict['function'] = "nseq"
    dataDict['data'] = nodeDict
    dataDict['classes'] = 'default'
    #print(elements, ' addNode ', elements.append(dataDict))
    elements.append(dataDict)
    return elements


def removeNode(node, elements):
    #print('before remove ', elements)
    for element in elements:
      if 'id' in element['data'].keys():
        if node['id'] == element['data']['id']:
            elements.remove(element)
    #print('after remove ', elements)
    return elements

def removeEdge(edge, elements):
    for element in elements:
        if 'source' in element['data'].keys():
            if (element['data']['source'] == edge['source']) and (element['data']['target'] == edge['target']):
                elements.remove(element)
    return elements

def addEdgeClasses(elements):
    for element in elements:
        if 'source' in element['data'].keys():
            element['classes'] = 'edge'
    return elements



def isEdgeInElements(edge, elements):
    for element in elements:
        if 'source' in element['data'].keys():
            if (element['data']['source'] == edge['source']) and (element['data']['target'] == edge['target']):
                return True
    return False

def addNewEdge(node1, node2, elements):
    edgeDict = {}
    dataDict = {}
    edgeDict['source'] = node1['id']
    edgeDict['target'] = node2['id']
    dataDict['data'] = edgeDict
    dataDict['classes'] = 'edge'
    if isEdgeInElements(edgeDict, elements):
        return elements
    else:
        elements.append(dataDict)
        return elements

def setNodeColorToselected(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['classes'] = 'selected'
    return elements

def setNodeFunctionToOR(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['data']['function'] = 'nor'
                element['data']['label'] = element['data']['label'][:-4] + '  or'
    return elements

def setNodeFunctionToXOR(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['data']['function'] = 'nxor'
                element['data']['label'] = element['data']['label'][:-4] + ' xor'
    return elements


def setNodeFunctionToAND(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['data']['function'] = 'nand'
                element['data']['label'] = element['data']['label'][:-4] + ' and'
    return elements


def setNodeFunctionToSEQ(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['data']['function'] = 'nseq'
                element['data']['label'] = element['data']['label'][:-4] + ' seq'
    return elements

def setIDColorToGreen(nodeID, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == nodeID:
                element['classes'] = 'green'
                #print("color set to green")
    return elements

def setIDColorToYellow(nodeID, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == nodeID:
                element['classes'] = 'yellow'
                #print("color set to yellow")
    return elements

def unselectNode(node, elements):
    for element in elements:
      if 'id' in element['data'].keys():
        if element['data']['id'] == node['id']:
            if 'classes' in element.keys():
                element['classes'] = 'default'
    return elements


def processTupleList(tupleList, elements):
    if (tupleList==None):
        return elements
    else:
        tl = tupleList.split('t(')
        #print(tl)
        del tl[0]
        #print(tl)
        for tupleT in tl:
            tuples = tupleT.split(',')
            #print(tuples)
            nodeID = tuples[0]
            indicator = tuples[1].split(")")[0]
            #print(nodeID, "   s   ", indicator)
            if (int(indicator) == 1):
                #print("node id is one")
                elements = setIDColorToGreen(str(int(nodeID)), elements)
            elif (int(indicator) == -1):
                #print("node id is minus one")
                elements = setIDColorToYellow(str(int(nodeID)), elements)
            #print("node indicator is ", int(indicator), ' node id is ', int(nodeID))
    return elements

app.layout = [
    dcc.Store(id='CurrentlySelectedNode'),
    dcc.Store(id='nodeSelectedForDeletion'),
    dcc.Store(id='selectedEdge'),
    dcc.Store(id='addEdgesFlag'),
    dcc.Store(id="functionFlag"),
    dcc.Store(id="sequenceFlag"),
    dcc.Store(id="realTimeFlag"),
    dcc.Store(id="config"),
    dcc.Store(id='Flags'),
    dcc.Store(id='edgeFlags'),

    dcc.Tabs([
        dcc.Tab(label="Edit Graph", children=[
            dcc.Input(id="newNodeNumber", type="number", value=None, placeholder="New Node Label"),
            html.Button(id='addNode_',children='Add a new Graph Node'),
            html.Button(id='RemoveNode', children='Remove a Specific Graph Node'),
            html.Button(id='RemoveEdge', children='Remove a Specific Graph Edge'),
            html.Button(id="addEdges", children="Start Adding Edges"),
            html.Button(id="functionToggle", children="Set Node Function"),
            html.Button(id="funcOR", children = "Set node to be an OR node" , style = dict(display='none')),
            html.Button(id="funcXOR", children = "Set node to be an XOR node" , style = dict(display='none')),
            html.Button(id="funcAND", children = "Set node to be an AND node" , style = dict(display='none')),
            html.Button(id="funcSEQ", children = "Set node to be an SEQUENTIAL node" , style = dict(display='none')),
            ]),
        dcc.Tab(label="Edit Sequence", children=[
            html.Button(id="sequenceToggle", children="Start Adding the Sequence"),
            html.H5(id="sequence"),
            dcc.Input(id="sequenceInput", type="number", value="None", placeholder="Input sequence")
            ]),
        dcc.Tab(label="Supervision", children=[
            html.Button(id="checkCorrectness", children="Check if the sequence conforms"),
            html.Button(id="makeStep", children="Make one step"),
            html.Button(id="realTime", children="Start Interactive Supervision"),
            html.H5(id="correctActionsLabel", children="Correct Actions: "),
            html.H5(id="correctActions"),
            html.H5(id="incorrectActionsLabel", children="Incorrect Actions: "),
            html.H5(id="incorrectActions"),
            html.H5(id="indicatorsLabel", children="indicators"),
            html.H5(id="indicators"),
            html.H5(id="correctness"),
            ]),
        dcc.Tab(label="Save/Upload Graph", children=[   
            html.Button("Download Graph", id="downloadBTN"),
            dcc.Download(id="download-graph"),
            dcc.Upload(id="upload", children=html.Button("Upload Graph", id = "uploadBTN")),
            ]),
        dcc.Tab(label="Examples", children=[
            
            ])
        ]),
    cyto.Cytoscape(
        id='cytoscape',
        elements=[
            {"data": {"id": "5", "label": "5 seq", "function": "nseq"}, "position": {"x": -159.93367616088958, "y": -20.399703591950203}}, {"data": {"id": "1", "label": "1 seq", "function": "nseq"}, "position": {"x": -227.66069208616423, "y": -21.215691735628212}}, {"data": {"id": "2", "label": "2  or", "function": "nor"}, "position": {"x": -31.823537603442315, "y": -22.847668022984223}}, {"data": {"id": "3", "label": "3 xor", "function": "nxor"}, "position": {"x": 79.55884400860577, "y": -17.543745089077163}}, {"data": {"id": "-5", "label": "S", "function": "nseq"}, "position": {"x": -286.872822676697, "y": -24.77054922503959}}, {"data": {"id": "8", "label": "8 seq", "function": "nseq"}, "position": {"x": -97.10258909768297, "y": -22.031679879306218}}, {"data": {"id": "6", "label": "6  or", "function": "nor"}, "position": {"x": 11.831828083331095, "y": -61.199110775850606}}, {"data": {"id": "4", "label": "4 seq", "function": "nseq"}, "position": {"x": 20.80769766378919, "y": 6.5279051494240665}}, {"data": {"id": "9", "label": "9 seq", "function": "nseq"}, "position": {"x": 78.93380481451139, "y": -96.53413021389244}}, {"data": {"id": "11", "label": "11 seq", "function": "nseq"}, "position": {"x": 147.44674477095305, "y": -17.13093242166292}}, {"data": {"id": "-7", "label": "F", "function": "nseq"}, "position": {"x": 222.45002354372266, "y": -13.221003408835257}}, {"classes": "edge", "data": {"source": "-5", "target": "1", "id": "a80d32ad-a9ea-42b9-828b-a2ffb6121af4"}}, {"classes": "edge", "data": {"source": "1", "target": "5", "id": "cd650965-95a2-4104-8d84-9946b6e08e00"}}, {"classes": "edge", "data": {"source": "5", "target": "8", "id": "ac4d5c1a-9e85-41e5-9f25-8f5ae7ded8a4"}}, {"classes": "edge", "data": {"source": "8", "target": "2", "id": "627a80f5-9034-4b49-87d3-3fa213524e13"}}, {"classes": "edge", "data": {"source": "2", "target": "4", "id": "8e97c6c2-bd82-482a-a089-3b8e8c7e41ca"}}, {"classes": "edge", "data": {"source": "4", "target": "3", "id": "27a1b5db-29c2-4d83-9046-487462bd234a"}}, {"classes": "edge", "data": {"source": "3", "target": "6", "id": "e226f22d-1509-409c-8a9d-6e4ba7c35010"}}, {"classes": "edge", "data": {"source": "6", "target": "2", "id": "e0b2e2b3-5086-4c9f-814a-13c5b50d9ef2"}}, {"classes": "edge", "data": {"source": "3", "target": "11", "id": "e5b72180-103a-4530-bb77-0e4d6524a0e9"}}, {"classes": "edge", "data": {"source": "11", "target": "-7", "id": "0cf008b1-90d0-4fd5-be7e-897ad3eba807"}}, {"classes": "edge", "data": {"source": "3", "target": "9", "id": "6242da7f-e53c-425f-aab6-3a1baec03a69"}}, {"classes": "edge", "data": {"source": "9", "target": "6", "id": "021bcd2b-5575-4f9e-8d83-4b7836a91af0"}}    
            ],
        layout={'name': 'preset'},
        style={'width': '100%', 'height': '400px'},
        stylesheet=[
            {
                'selector': 'default',
                'style': {
                    'background-color': '#969696',
                    'content': 'data(label)'
                }
            },
            {
                'selector': '.edge',
                'style': {
                    'target-arrow-shape': 'triangle',
                    'arrow-scale': "2",
                    'curve-style': 'bezier',
                }
            },
            {
                'selector': '.default',
                'style': {
                    'background-color': '#969696',
                    'content': 'data(label)',
                    #'content': 'data(function)',
                }
            },
            {
                'selector': '.selected',
                'style': {
                    'background-color': '#0099ff',
                    'content': 'data(label)'
                }
            },
            {
                'selector': '.green',
                'style': {
                    'background-color': '#008000',
                    'content': 'data(label)'
                }
            },
            {
                'selector': '.yellow',
                'style': {
                    'background-color': '#FFFF00',
                    'content': 'data(label)'
                }
            }
        ]
    ),
]

#"Add Node" button callback    
@callback(
    Output("cytoscape", "elements",  allow_duplicate=True),
    Output("newNodeNumber", "value", allow_duplicate=True),
    State("cytoscape", "elements"),
    Input("addNode_", "n_clicks"),
    State("newNodeNumber", "value"),
    prevent_initial_call=True,
)
def addNewNodeCallback(cyto, n_cl, value):
    if value == None:
        return addNewNode(cyto), None
    else:
        return addNewNodeNumber(cyto, value), None

#Callback for adding edges by tapping two nodes
@callback(
    Output("Flags", "data", allow_duplicate=True),
    Output("RemoveNode", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("CurrentlySelectedNode", "data", allow_duplicate=True),
    Output('nodeSelectedForDeletion', "data", allow_duplicate=True),
    Output("sequence", "children", allow_duplicate=True),
    Output("funcOR", "style", allow_duplicate=True),
    Output("funcXOR", "style", allow_duplicate=True),
    Output("funcAND", "style", allow_duplicate=True),
    Output("funcSEQ", "style", allow_duplicate=True),
    Output("correctActions", "children", allow_duplicate=True),
    Output("incorrectActions", "children", allow_duplicate=True),
    Output("correctness", "children", allow_duplicate=True),
    Output("config", "data", allow_duplicate=True),
    State("CurrentlySelectedNode", "data"),
    State("cytoscape", "elements"),
    State("RemoveNode", "children"),
    Input("cytoscape", "tapNodeData"),
    State("Flags", "data"),
    State("addEdgesFlag", "data"),
    State("nodeSelectedForDeletion", "data"),
    State("sequenceFlag", "data"),
    State("sequence", "children"),
    State("functionFlag", "data"),
    State("realTimeFlag", "data"),
    State("correctActions", "children"),
    State("incorrectActions","children"),
    State("config", "data"),
    prevent_initial_call=True,
)
def tapNodeAddEdge(prevNode, elements, rmNodeButton, tappedNode, flags, addEdgesFlag, nodeSelectedForDeletion, seqFlag, sequence, functionFlag, realTime, correct, incorrect, config):
    if (realTime != None) and (realTime == True):
        if (config != None):
            config = makeStep( "a(" + tappedNode['id'] + ") | " + getTupleList(config) + " | " + getCorrectList(config) + " | " + getIncorrectList(config) + " | " + elementsToGraph(elements) + " | " + getStatus(config) + " " )
            if ("-7" in getCorrectList(config)):
                config = makeStep(config)
        else:
            config = makeStep(generateTerm(elements,[tappedNode['id']]))
        return flags, rmNodeButton,  processTupleList(getTupleList(config), elements), None, None, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), getCorrectList(config), getIncorrectList(config), getStatus(config), config
    else:
        if (functionFlag !=None) and (functionFlag == True):
            store = tappedNode
            funcButton = dict(display='initial')
            return flags, rmNodeButton, elements, store, nodeSelectedForDeletion, sequence, funcButton, funcButton, funcButton, funcButton, None, None, None, config
        else:
            if (seqFlag != None) and (seqFlag == True):
                if sequence == None:
                    sequence = tappedNode['id']
                else:
                    sequence = sequence + "," + tappedNode['id']
                return flags, rmNodeButton, elements, None, nodeSelectedForDeletion, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, None, None, config
            else:
                if (addEdgesFlag == None) or (addEdgesFlag == False):
                    store = None
                    if (flags != None) and ('removeNode' in flags.keys()) and (flags['removeNode'] == True):
                        elements = removeNode(tappedNode, elements)
                        flags['removeNode'] = False
                        rmNodeButton = "Remove a Specific Graph Node"
                        return flags, rmNodeButton, elements, store, nodeSelectedForDeletion, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, None, None, config
                    elif nodeSelectedForDeletion == None:
                        nodeSelectedForDeletion = tappedNode
                        elements = setNodeColorToselected(tappedNode, elements)
                        rmNodeButton = "Remove the Selected Graph Node"
                    else:
                        elements = unselectNode(tappedNode, elements)
                        elements = unselectNode(nodeSelectedForDeletion, elements)
                        nodeSelectedForDeletion = None
                        rmNodeButton = "Remove a Specific Graph Node"
                    return flags, rmNodeButton, elements, None, nodeSelectedForDeletion, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, None, None, config
                else:
                    #print(elements)
                    store = None
                    #print(flags)
                    if (flags != None) and ('removeNode' in flags.keys()) and (flags['removeNode'] == True):
                            elements = removeNode(tappedNode, elements)
                            flags['removeNode'] = False
                            rmNodeButton = "Remove a Specific Graph Node"
                            return flags, rmNodeButton, elements, store, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, None, None, config
                    else:
                        if prevNode == None:
                            store = tappedNode
                            elements = setNodeColorToselected(tappedNode, elements)
                            rmNodeButton = "Remove the Selected Graph Node"
                        else:
                            elements = unselectNode(prevNode, elements)
                            elements = addNewEdge(prevNode, tappedNode, elements)
                            elements = unselectNode(tappedNode, elements)
                            store = None   
                            rmNodeButton = "Remove a Specific Graph Node"
                    return flags, rmNodeButton, elements, store, nodeSelectedForDeletion, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, None, None, config

#"Remove Node" buttton callback
@callback(
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("Flags", "data", allow_duplicate=True),
    Output("RemoveNode", "children", allow_duplicate=True),
    Output("CurrentlySelectedNode", "data", allow_duplicate=True),
    Output('nodeSelectedForDeletion', "data", allow_duplicate=True),
    State("cytoscape", "elements"),
    State("Flags", "data"),
    State("CurrentlySelectedNode", "data"),
    Input("RemoveNode", "n_clicks"),
    State("RemoveNode", "children"),
    State("nodeSelectedForDeletion", "data"),
    prevent_initial_call=True,
    )
def removeNodePressed(elements, flags, selectedNode, clicks, rmNodeButton, nodeSelectedForDeletion):
    if nodeSelectedForDeletion != None:
        elements = removeNode(nodeSelectedForDeletion, elements)
        rmNodeButton = "Remove a Specific Graph Node"
        nodeSelectedForDeletion = None
    elif selectedNode != None:
        elements = removeNode(selectedNode, elements)
        rmNodeButton = "Remove a Specific Graph Node"
        selectedNode = None
    else:
        dictF = {}
        dictF['removeNode'] = True
        flags = dictF
        rmNodeButton = "Select Graph Node To Remove"
    return elements, flags, rmNodeButton, selectedNode, nodeSelectedForDeletion

#"Remove Edge" button callback
@callback(
    Output("RemoveEdge", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("edgeFlags", "data", allow_duplicate=True),
    Output("selectedEdge", "data", allow_duplicate=True),
    Input("RemoveEdge", "n_clicks"),
    State("edgeFlags", "data"),
    State("cytoscape", "elements"),
    State("selectedEdge", "data"),
    prevent_initial_call=True,
)
def removeEdgeButton(button, edgeFlags, elements, selectedEdge):
    if (edgeFlags == None) or (edgeFlags == False):
        if (selectedEdge == None):
            flag = True
            edgeFlags = flag
            buttonMSG = "Select Edge to be Removed"
        else:
            elements = removeEdge(selectedEdge, elements)
            selectedEdge = None
            buttonMSG = "Remove a Specific Graph Edge"
    else:
        edgeFlags = False
        buttonMSG = "Remove a Specific Graph Edge"
    return buttonMSG, elements, edgeFlags, selectedEdge

#Callback on tapped/clicked edge
@callback(
    Output("RemoveEdge", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("edgeFlags", "data", allow_duplicate=True),
    Output("selectedEdge", "data", allow_duplicate=True),
    State("edgeFlags", "data"),
    State("cytoscape", "elements"),
    Input("cytoscape", "tapEdgeData"),
    prevent_initial_call=True,
)
def selctedEdge(flags, elements, edge):
    selectedEdge = None
    if (flags != None) and (flags == True):
        elements = removeEdge(edge, elements)
        selectedEdge = None
        buttonMSG = "Remove a Specific Graph Edge"
        flags = False
    elif (flags == None) or (flags == False):
        selectedEdge = edge
        buttonMSG = "Remove Selected Edge"
    return buttonMSG, elements, flags, selectedEdge


#Button to start/stop adding edges
@callback(
    Output("addEdgesFlag", "data"),
    Output("addEdges", "children"),
    Input("addEdges", "n_clicks"),
    State("addEdgesFlag", "data"),
    prevent_initial_call=True,
)
def edgeAddButtonPressed(clicks, edgesToggle):
    if (edgesToggle==None) or (edgesToggle==False):
        edgesToggle = True
        buttonMSG = "Stop Adding Edges"
    elif(edgesToggle == True):
        edgesToggle = False
        buttonMSG = "Start Adding Edges"
    return edgesToggle, buttonMSG

#callback to download the list of elements as a json
@callback(
    Output("download-graph", "data"),
    Input("downloadBTN", "n_clicks"),
    State("cytoscape", "elements"),
    State("sequence","children"),
    prevent_initial_call=True,
)
def downloadGraphButtonPushed(clicks, elements, sequence):
    sdict = {}
    sdict['sequence'] = sequence
    seqDict = {}
    seqDict['data']= sdict
    elements.append(seqDict)
    jsonElements = json.dumps(elements)
    #print(jsonElements)
    return {'content': jsonElements, 'filename': 'graph.json'}

#callback to upload a list of elements as a json
@callback(
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("sequence", "children", allow_duplicate=True),
    Input("upload", "contents"),
    prevent_initial_call=True,
)
def uploadGraph(content):
    #print(content)
    content = base64.b64decode(content.split(',')[1])
    sequence = json.loads(content)[-1]['data']['sequence']
    return (addEdgeClasses((json.loads(content)[:-1]))), sequence
    
@callback(
    Output("correctness", "children", allow_duplicate=True),
    State("cytoscape", "elements"),
    Input("checkCorrectness", "n_clicks"),
    State("sequence","children"),
    prevent_initial_call=True,
)
def checkCorrectnessButton(elements,clicks,sequence):
    #tempList = [1,2,3,4,5,6]
    tempList = sequence.split(",")
    #print(tempList)
    correct = checkCorrectness(generateTerm(elements,tempList))
    if correct:
        return "Correct sequence"
    else:
        return "Incorrect sequence"

@callback(
    Output("sequenceFlag", "data", allow_duplicate=True),
    Output("sequenceToggle", "children", allow_duplicate=True),
    Input("sequenceToggle", "n_clicks"),
    State("sequenceFlag", "data"),
    State("sequenceToggle", "children"),
    prevent_initial_call=True,
)
def addSequenceButton(clicks, flag, button):
    if ((flag == None) or (flag == False)):
        flag = True
        button = "Finish adding sequence"
    else:
        flag = False
        button = "Start adding the sequence"
    return flag, button

@callback(
    Output("correctActions", "children", allow_duplicate=True),
    Output("incorrectActions", "children", allow_duplicate=True),
    Output("config", "data", allow_duplicate=True),
    Output("indicators", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Input("makeStep", "n_clicks"),
    State("correctActions", "children"),
    State("incorrectActions", "children"),
    State("cytoscape","elements"),
    State("sequence", "children"),
    State("config", "data"),
    State("indicators", "children"),
    prevent_initial_call=True,
)
def makeStepButton(click,correctList,incorrectList, elements, sequence, config, indicators):
    if (config == None):
        config = makeStep(generateTerm(elements,sequence.split(",")))
    else:
        config = makeStep(config)
    if (config != None):
        return getCorrectList(config), getIncorrectList(config), config, getTupleList(config), processTupleList(getTupleList(config), elements)
    else:
        return "execution finished " + correctList, incorrectList, config, indicators, elements
    
    
#button setFunction
@callback(
    #Output("cytoscape", "elements", allow_duplicate=True),
    Output("functionFlag", "data", allow_duplicate=True),
    Output("functionToggle", "children", allow_duplicate=True),
    Input("functionToggle", "n_clicks"),
    Input("functionFlag", "data"),
    prevent_initial_call = True, 
)
def buttonSetFunc(clicks, flag):
    if (flag == None) or (flag == False):
        flag = True
        return flag, "Finish setting node functions"
    else:
        flag = False
        return flag, "Start setting node functions"
    
#function button Pressed
@callback(
    Output("funcOR", "style", allow_duplicate=True),
    Output("funcXOR", "style", allow_duplicate=True),
    Output("funcAND", "style", allow_duplicate=True),
    Output("funcSEQ", "style", allow_duplicate=True),
    Output("CurrentlySelectedNode", "data", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    State("CurrentlySelectedNode", "data"),
    Input("funcOR", "n_clicks"),
    Input("funcXOR", "n_clicks"),
    Input("funcAND", "n_clicks"),
    Input("funcSEQ", "n_clicks"),
    State("cytoscape", "elements"),
    prevent_initial_call = True,
)
def funButton(selectedNode, f_or, f_xor, f_and, f_seq, elements):
    if (selectedNode != None):
        button = ctx.triggered_id
        #print(button)
        match button:
            case "funcOR":
                elements = setNodeFunctionToOR(selectedNode, elements)
            case "funcXOR":
                elements = setNodeFunctionToXOR(selectedNode, elements)
            case "funcAND":
                elements = setNodeFunctionToAND(selectedNode, elements)
            case "funcSEQ":
                elements = setNodeFunctionToSEQ(selectedNode, elements)
        #print(elements)
        return dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, elements
    else:
        return dict(display='default'), dict(display='default'), dict(display='default'), dict(display='default'), None, elements
    

#button realtime
@callback(
    #Output("cytoscape", "elements", allow_duplicate=True),
    Output("realTimeFlag", "data", allow_duplicate=True),
    Output("realTime", "children", allow_duplicate=True),
    Input("realTime", "n_clicks"),
    Input("realTimeFlag", "data"),
    prevent_initial_call = True, 
)
def buttonSetFunc(clicks, flag):
    if (flag == None) or (flag == False):
        flag = True
        return flag, "Stop Interactive Supervision"
    else:
        flag = False
        return flag, "Start Interactive Supervision"
    
        
#app.run(debug=True)
app.run(debug=True, host= "192.168.0.102", port=8053)


In [4]:
import maude


def initializeMaude():
    maude.init()
    maude.load('graphCookies.maude')
    module = maude.getCurrentModule()
    return module


def checkCorrectness(t):
    maude.init()
    maude.load('graphCookies.maude')
    module = maude.getCurrentModule()
    #module = initializeMaude()
    term = module.parseTerm(t)
    correctPattern = module.parseTerm('SL:ActionList | TL:TupleList | CL:ActionList | WL:ActionList | G:Graph | correct')
    for r, sb, ctx, rl in term.search(maude.ANY_STEPS, correctPattern):
        #print(r)
        return True
    return False


def checkIncorrectness(t):
    module = initializeMaude()
    term = module.parseTerm(t)
    incorrectPattern = module.parseTerm('SL:ActionList | TL:TupleList | CL:ActionList | WL:ActionList | G:Graph | incorrect')
    for r, sb, ctx, rl in term.search(maude.ANY_STEPS, incorrectPattern):
        return True
    return False

def makeStep(termString):
    #print(termString)
    module = initializeMaude()
    term = module.parseTerm(termString)
    pattern = module.parseTerm('SL:ActionList | TL:TupleList | CL:ActionList | WL:ActionList | G:Graph | S:Status')
    for r, sb, ctx, rl in term.search(0, pattern):
        return str(r)

def elementsToGraph(elements):
    graph = ""
    for element in elements:
        if 'source' in element['data'].keys():
            src = element['data']['source']
            dst = element['data']['target']
            graph = graph + '(n[' + src + ']: ' + getFunctionByID(src, elements) + ') -> (n[' + dst + ']: '+  getFunctionByID(dst, elements) +') ;; '
    return str(graph[:-4])

def getFunctionByID(ID, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == ID:
                return element['data']['function']
           
def listToSequence(slist):
    seq = "("
    for element in slist:
        seq = seq + 'a(' + str(element) + ') ; '
    seq = seq[:-3] + ")"
    return str(seq)

def getInputList(term):
    return term.split('|')[0]
def getTupleList(term):
    return term.split('|')[1]
def getCorrectList(term):
    return term.split('|')[2]
def getIncorrectList(term):
    return term.split('|')[3]
def getStatus(term):
    return term.split('|')[5]

def generateTerm(elements, actions):
    return "" + listToSequence(actions) + " | nil | nil | nil | " + elementsToGraph(elements) + " | starting"


#print(checkIncorrectness(generateTerm(exelements, eslist)))
e = [
            {'data': {'id': '5', 'label': 'escape'}},
            {'data': {'id': '1', 'label': '1'}}, 
            {'data': {'id': '2', 'label': '2'}}, 
            {'data': {'id': '3', 'label': '3'}},
            {'data': {'source': '1', 'target': '2'}}, 
            {'data': {'source': '2', 'target': '3'}}
        ]
print(processTupleList("t(1, -1), t(2, 1)", e))


[{'data': {'id': '5', 'label': 'escape'}}, {'data': {'id': '1', 'label': '1'}, 'classes': 'yellow'}, {'data': {'id': '2', 'label': '2'}, 'classes': 'green'}, {'data': {'id': '3', 'label': '3'}}, {'data': {'source': '1', 'target': '2'}}, {'data': {'source': '2', 'target': '3'}}]
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[3], line 530, in uploadGraph(
    content=b'[{"data": {"id": "5", "label": "5 seq", "funct...e-f0190ed141aa"}}, {"data": {"sequence": null}}]'
)
    528 content = base64.b64decode(content.split(',')[1])
    529 sequence = json.loads(content)[-1]['data']['sequence']
--> 530 return addEdgeClass(json.loads(content)[:-1]), sequence
        content = b'[{"data": {"id": "5", "label": "5 seq", "function": "nseq"}, "position": {"x": -156.78239896223306, "y": -20.399703591950203}, "classes": "green"}, {"data": {"id": "1", "label": "1 seq", "function": "nseq"}, "po